# Rubric: (accounts for 20% of the final grade.)

* Choose data of your choice, (labelled data) 1%

* Preprocessing and Data Cleansing 1%

* Feature Engineering 2%

* Use SVM, Random Forest, Naïve Bayes, k-Nearest Neighbor, XG-Boost, SGD etc. 2%

* (explore classifications using either BERT or XLNET 1%)

* Perform Evaluations, 2%

* Compare and decide which algorithm is performing as the champion model 0.5%

* Perform Error Analysis, 2%

* Perform Visualizations, Graph the results 2%

* One or Two pages of presentation 0.5%

* The report, detail explanations 2%

* Analysis of Bias and Variability 2%

* Identifying, measuring and control the machine’s thresholds of factors of prediction hardship 1%

* Group work: 1%

# Preparing And Preprocessing

In [1]:
!pip install nltk
!pip install tensorflow_text
!pip install --upgrade tensorflow-text
!pip install transformers
!pip install --upgrade tensorflow

import pandas as pd
import random
import re
import requests
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import string, time
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 54.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
# This variable uses in findall() function of a regular expression to get required samples.
sampleRegex = r"(?<!\S)[A-Za-z]+(?!\S)|(?<!\S)[A-Za-z]+(?=:(?!\S))"

# Load English stopwords
stop_words = set(stopwords.words('english'))

# Initialize WordNetLemmatizer and PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
#porter_stemmer = PorterStemmer()

# This function takes two parameters (list of samples and any string). It drops this string from list.
def cleanedListFun(List, st):
    lst = [x for x in List if str(x) != st]
    return lst

# This function takes any path book and returns data partitions and its title.
def TextDataPartitioning(url=""):
    response = requests.get(url)
    txtBook = response.text
    # Find all required samples in txtBook by using sampleRegex and return the result in list.
    lst = re.findall(sampleRegex, txtBook)
    # drop "NaN" and 'Unnamed' from List
    cleanedList = cleanedListFun(lst, 'NaN')
    cleanedList = cleanedListFun(cleanedList, 'Unnamed')

    stIndex = []
    lstPartitions = []
    random.seed(42)
    # divide List into 200 partitions, each partition contains 100 random samples.
    for i in range(0, 200):
        # randint () function puts random number in variable from particular range.
        l = random.randint(0, len(cleanedList) - 100)
        # This option cancels repeating the same partition.
        if l not in stIndex:
            stIndex.append(l)
            lstPartitions.append(cleanedList[l:l + 100])
        else:
            lstPartitions.append(cleanedList[l - 1:l + 99])
    
    # Remove stopwords and perform lemmatization and stemming on each partition
    processed_partitions = []
    for partition in lstPartitions:
        filtered_partition = [word for word in partition if word.lower() not in stop_words]
        lemmatized_partition = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in filtered_partition]
       # stemmed_partition = [porter_stemmer.stem(word) for word in lemmatized_partition]
        processed_partitions.append(lemmatized_partition)
    
    # create object dataframe from pandas library to put partitions
    df = pd.DataFrame()
    df["Paragraphs"] = processed_partitions
    return df

In [3]:
books = {
    'Pride and Prejudice': {
        'author': 'Jane Austen',
        'url': 'https://www.gutenberg.org/cache/epub/1342/pg1342.txt'
    },
    'Anna Karenina': {
        'author': 'Leo Tolstoy',
        'url': 'https://www.gutenberg.org/cache/epub/1399/pg1399.txt'
    },
    'Little Women or Meg, Jo, Beth, and Amy': {
        'author': 'Louisa M. Alcott',
        'url': 'https://www.gutenberg.org/cache/epub/37106/pg37106.txt'
    },
    'Wuthering Heights': {
        'author': 'Emily Brontë',
        'url': 'https://www.gutenberg.org/cache/epub/768/pg768.txt'
    },
    'This Side of Paradise': {
        'author': 'F. Scott Fitzgerald',
        'url': 'https://www.gutenberg.org/files/805/805-0.txt'
    }
}

FullData = pd.DataFrame(columns=[ 'Author'])

for book, details in books.items():
    author = details['author']
    url = details['url']
    
    # Fetch and process the text data from the URL using TextDataPartitioning function
    text_data = TextDataPartitioning(url)
    
    # Add book, author, nationality, and text data to the DataFrame
    text_data['Author'] = author
    
    # Append to the FullData DataFrame
    FullData = FullData.append(text_data, ignore_index=True)

display(FullData)


<ipython-input-3-4f2eea610fec>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  FullData = FullData.append(text_data, ignore_index=True)
<ipython-input-3-4f2eea610fec>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  FullData = FullData.append(text_data, ignore_index=True)
<ipython-input-3-4f2eea610fec>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  FullData = FullData.append(text_data, ignore_index=True)
<ipython-input-3-4f2eea610fec>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  FullData = FullData.append(text_data, ignore_index=True)
<ipython-input-3-4f2eea610fec>:37: FutureWarning: The frame.append method is deprecated and 

,Author,Paragraphs
0,Jane Austen,"[advantage, Miss, Lydia, Bennet, come, upon, h..."
1,Jane Austen,"[unaffectedly, civil, force, younger, sister, ..."
2,Jane Austen,"[give, one, two, early, speeches, slight, come..."
3,Jane Austen,"[report, engagement, could, Elizabeth, loss, t..."
4,Jane Austen,"[higher, others, solidity, reflections, often,..."
...,...,...
995,F. Scott Fitzgerald,"[tell, worry, seem, progress, perfectly, Amory..."
996,F. Scott Fitzgerald,"[lately, reread, Aeschylus, divine, irony, fin..."
997,F. Scott Fitzgerald,"[well, know, little, man, laugh, conscientious..."
998,F. Scott Fitzgerald,"[afterward, repeat, anecdotes, life, could, ma..."


In [4]:
df = pd.DataFrame(FullData)
df.to_csv('partitions.csv', index=False)

In [5]:
# Convert 'Paragraphs' column to string type
df['Paragraphs'] = df['Paragraphs'].astype(str)

# Using the str.lower() method
df['Paragraphs'] = df['Paragraphs'].str.lower()

# Using the apply() method
df['Paragraphs'] = df['Paragraphs'].apply(lambda x: x.lower())
df


,Author,Paragraphs
0,Jane Austen,"['advantage', 'miss', 'lydia', 'bennet', 'come..."
1,Jane Austen,"['unaffectedly', 'civil', 'force', 'younger', ..."
2,Jane Austen,"['give', 'one', 'two', 'early', 'speeches', 's..."
3,Jane Austen,"['report', 'engagement', 'could', 'elizabeth',..."
4,Jane Austen,"['higher', 'others', 'solidity', 'reflections'..."
...,...,...
995,F. Scott Fitzgerald,"['tell', 'worry', 'seem', 'progress', 'perfect..."
996,F. Scott Fitzgerald,"['lately', 'reread', 'aeschylus', 'divine', 'i..."
997,F. Scott Fitzgerald,"['well', 'know', 'little', 'man', 'laugh', 'co..."
998,F. Scott Fitzgerald,"['afterward', 'repeat', 'anecdotes', 'life', '..."


In [6]:
exclude = string.punctuation
def remove_punct(text):
    for char in exclude:
        text = text.replace(char, '')
    return text
df['Paragraphs'].apply(remove_punct)

0      advantage miss lydia bennet come upon happiest...
1      unaffectedly civil force younger sister civil ...
2      give one two early speeches slight come seriou...
3      report engagement could elizabeth loss till re...
4      higher others solidity reflections often strik...
                             ...                        
995    tell worry seem progress perfectly amory lose ...
996    lately reread aeschylus divine irony find answ...
997    well know little man laugh conscientious stop ...
998    afterward repeat anecdotes life could make sou...
999    every one sew bag rest college amory find writ...
Name: Paragraphs, Length: 1000, dtype: object

In [7]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

df['Paragraphs'] = df['Paragraphs'].apply(remove_special_characters)
df

,Author,Paragraphs
0,Jane Austen,advantage miss lydia bennet come upon happiest...
1,Jane Austen,unaffectedly civil force younger sister civil ...
2,Jane Austen,give one two early speeches slight come seriou...
3,Jane Austen,report engagement could elizabeth loss till re...
4,Jane Austen,higher others solidity reflections often strik...
...,...,...
995,F. Scott Fitzgerald,tell worry seem progress perfectly amory lose ...
996,F. Scott Fitzgerald,lately reread aeschylus divine irony find answ...
997,F. Scott Fitzgerald,well know little man laugh conscientious stop ...
998,F. Scott Fitzgerald,afterward repeat anecdotes life could make sou...


In [8]:
label_encoder = LabelEncoder()
df.Author = label_encoder.fit_transform(df.Author)


In [9]:
X_train, X_val, y_train, y_val = train_test_split(df.Paragraphs, 
                                                  df.Author, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.Author)

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [11]:
def preprocess_text(text, max_seq_length):
    # Tokenize the text
    tokens = tokenizer.tokenize(text)
    # Add special tokens [CLS] and [SEP]
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    
    # Convert tokens to input IDs
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    
    # Pad or truncate the input IDs to the desired sequence length
    input_ids = input_ids[:max_seq_length]
    padding_length = max_seq_length - len(input_ids)
 
    input_ids = input_ids + [0] * padding_length
   

    # Create the attention mask
    attention_mask = [1] * len(input_ids)
    
    # Create the token type IDs
    token_type_ids = [0] * len(input_ids)
    
    return input_ids, attention_mask , token_type_ids

In [12]:
def preprocess_df(x_df , y_df):

  max_seq_length = 100  # Set the desired maximum sequence length

  X_df_word_ids = []  # List to store preprocessed input IDs
  attention_masks = []  # List to store attention masks
  token_type_ids = []  # List to store token type IDs

  for text in x_df:
    input_ids, attention_mask , token_type_id= preprocess_text(text, max_seq_length)
    X_df_word_ids.append(input_ids)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)

  # Convert the lists to TensorFlow tensors
  X_df_word_ids = tf.convert_to_tensor(X_df_word_ids)
  attention_masks = tf.convert_to_tensor(attention_masks)
  token_type_ids = tf.convert_to_tensor(token_type_ids)

  y_train = y_df  # Your training labels
  return X_df_word_ids , attention_masks , token_type_ids , y_train

In [13]:
max_seq_length = 100
input_word_ids = tf.keras.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_word_ids")
input_mask = tf.keras.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_mask")
input_type_ids = tf.keras.Input(shape=(max_seq_length,), dtype=tf.int32, name="input_type_ids")


In [14]:
bert_model = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"  # BERT base model
bert_layer = hub.KerasLayer(bert_model, trainable=False)

In [15]:
bert_output = bert_layer({"input_word_ids": input_word_ids, "input_mask": input_mask, "input_type_ids": input_type_ids})
dropout = tf.keras.layers.Dropout(0.5)(bert_output["pooled_output"])
output = tf.keras.layers.Dense(5, activation='softmax')(dropout)

model = Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_mask (InputLayer)        [(None, 100)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 input_word_ids (InputLayer)    [(None, 100)]        0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'pooled_output': (  109482241   ['input_mask[0][0]',             
                                None, 768),                       'input_type_ids[0][0]',     

In [17]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='categorical_crossentropy',
 metrics=METRICS)

In [18]:
X_train_word_ids, attention_masks_train, token_type_ids_train, y_train = preprocess_df(X_train, y_train)

In [19]:
# Encode the true labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Convert the encoded labels to one-hot encoded format
y_train_onehot = to_categorical(y_train_encoded, num_classes=5)
y_val = label_encoder.fit_transform(y_val)
y_val = to_categorical(y_val, num_classes=5)

In [20]:
model.fit([X_train_word_ids, attention_masks_train, token_type_ids_train],y_train_onehot,  batch_size = 16 , epochs=30)

Epoch 1/30
54/54 [==============================] - 19s 113ms/step - loss: 1.8249 - accuracy: 0.7835 - precision: 0.2464 - recall: 0.0400
Epoch 2/30
54/54 [==============================] - 6s 116ms/step - loss: 1.7517 - accuracy: 0.7828 - precision: 0.1983 - recall: 0.0282
Epoch 3/30
54/54 [==============================] - 6s 118ms/step - loss: 1.7013 - accuracy: 0.7873 - precision: 0.3200 - recall: 0.0565
Epoch 4/30
54/54 [==============================] - 6s 117ms/step - loss: 1.6694 - accuracy: 0.7894 - precision: 0.3077 - recall: 0.0424
Epoch 5/30
54/54 [==============================] - 6s 117ms/step - loss: 1.6606 - accuracy: 0.7929 - precision: 0.3636 - recall: 0.0471
Epoch 6/30
54/54 [==============================] - 6s 117ms/step - loss: 1.6200 - accuracy: 0.7932 - precision: 0.3535 - recall: 0.0412
Epoch 7/30
54/54 [==============================] - 6s 118ms/step - loss: 1.6230 - accuracy: 0.7904 - precision: 0.2970 - recall: 0.0353
Epoch 8/30
54/54 [======================

In [21]:
# Assuming you have the validation data and preprocessed inputs: X_val_word_ids, attention_masks_val, token_type_ids_val, y_val_encoded
X_val_word_ids, attention_masks_val, token_type_ids_val, y_val_encoded = preprocess_df(X_val, y_val)
# Evaluate the model on the validation data
results = model.evaluate([X_val_word_ids, attention_masks_val, token_type_ids_val], y_val_encoded)

# Print the loss and metric values
print("Loss: ", results[0])
print("Accuracy: ", results[1])
print("Precision: ", results[2])
print("Recall: ", results[3])


5/5 [==============================] - 2s 210ms/step - loss: 1.3875 - accuracy: 0.8040 - precision: 0.8000 - recall: 0.0267
Loss:  1.3874684572219849
Accuracy:  0.8040000200271606
Precision:  0.800000011920929
Recall:  0.02666666731238365
